In [1]:
import numpy as np
from __future__ import division
import pandas as pd 
np.set_printoptions(suppress=True)

In [4]:
class CorticalCell(object):
    """
        constructs cortical cell instance/object. 
        num_dendrites: int used to specify how many dendrites your cell will have
        
        size_of_vector: int used to specify size of input encoding vector. Once given this value the dendrites will
        make random connections to the indicies of this vector.
    """

    def __init__(self, cell_num, num_dendrites, size_of_vector):
        self.cell_num = cell_num
        self.num_dendrites = num_dendrites
        self.size_of_vector = size_of_vector
    
    
    
    def Dendrite_connections(self,percent_of_input_space):
        
        """Randomly connencts dendrites to input space. Assigns each dendrite an indice from a subset of the
        input space"""
        
        subset_size = int(percent_of_input_space*size_of_vector)
        
        choice_pool = np.random.randint(size_of_vector, size = subset_size)
        
        return  np.random.choice(choice_pool, self.num_dendrites)
        
        #np.random.randint(size_of_vector, size=self.num_dendrites)
    
   
        
    def Synapse_perm(self, mu, sigma,percent_of_input_space):
        
        """Calls Dendrite_connection function, and then assigns synaptic permances to each dendrite centered around
        a normal distribution determined by mean(mu) and standard deviation(sigma)"""
            
        D_connections = self.Dendrite_connections(percent_of_input_space)
        D_length = len(D_connections)
        
        
        #mu, sigma = 0.5, 0.4 # mean and standard deviation
        synapse_strength = np.clip((sigma * np.random.randn(D_length) + mu),0,1)
        return np.vstack((D_connections,synapse_strength))
        
        
      
        

In [3]:
def winner_cell_cutoff(cell_dendrite_num,top_percent):
    sorted_array = -np.sort(-cell_dendrite_num)
    size_of_array = len(sorted_array)
    top_indice = int(top_percent*size_of_array)
    return sorted_array[top_indice]
    

In [4]:
def adjust_synapse_strengths(winner_cells, synapse_inc, synapse_dec, synaptic_perm_threshold):
    syn_inc = 1+synapse_inc
    syn_dec = 1-synapse_dec
    
    for cell in winner_cells:
        synapses = cell.Dendrites_and_Synapses[1,:]
        connected_synapses = np.flatnonzero(synapses > synaptic_perm_threshold)
        boosted_synapses =  np.multiply(cell.Dendrites_and_Synapses[1,connected_synapses], syn_inc)
        boosted_synapses = np.clip(boosted_synapses,0,1)
        cell.Dendrites_and_Synapses[1,connected_synapses] = boosted_synapses
        non_connected_synapses = np.flatnonzero(synapses < synaptic_perm_threshold)
        decrease_synapses = np.multiply(cell.Dendrites_and_Synapses[1,non_connected_synapses], syn_dec)
        decrease_synapses = np.clip(decrease_synapses,0,1)
        cell.Dendrites_and_Synapses[1,non_connected_synapses] = decrease_synapses
    return winner_cells

In [5]:
cortex = np.empty(1000,dtype=object)
num_dendrites = 20
size_of_vector = 400
mu = 0.5 #mean
sigma = 0.4 #standard deviation
potentional_percent = 0.85 #percent that a cells dendrite may connect to

for cortex_cell in range(1000):
    cortex[cortex_cell] = CorticalCell(cortex_cell, num_dendrites, size_of_vector)
    cortex[cortex_cell].Dendrites_connections = cortex[cortex_cell].Dendrite_connections(potentional_percent)
    cortex[cortex_cell].Dendrites_and_Synapses = cortex[cortex_cell].Synapse_perm(mu,sigma,potentional_percent)

In [6]:
cortex[49].Dendrites_and_Synapses

array([[ 193.        ,   87.        ,  136.        ,  177.        ,
         123.        ,   21.        ,  164.        ,  351.        ,
         374.        ,  293.        ,  132.        ,  238.        ,
         272.        ,   21.        ,  263.        ,  363.        ,
         354.        ,  164.        ,  310.        ,  355.        ],
       [   0.91646372,    0.77363576,    0.        ,    0.31715941,
           0.93211621,    0.        ,    0.25055073,    0.86670466,
           0.68077916,    0.2335472 ,    1.        ,    0.52083672,
           0.93619911,    0.8932645 ,    1.        ,    0.70971483,
           0.        ,    0.29854399,    1.        ,    0.46934903]])

In [7]:
sample_input = np.random.randint(2, size=(100, 400))

In [8]:
sample_input 

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [1, 0, 1, ..., 0, 1, 0],
       ..., 
       [1, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 1, 0]])

In [9]:
#def Learning(input, cortex, top_conncetion_percent, synaptic_perm_threshold):
top_conncetion_percent = .10
synaptic_perm_threshold = 0.5
cell_dendrite_num = np.zeros(len(cortex))
#cell_dendrite_num: array of zeros constructed to keep track of number of dendrites connected to each
#input encoding(on bits)

for sample in sample_input:
    for i in range(len(cortex)): 
        #each cell dendries are randomly connected to input array. The input indicies
        #are returned to array "dendrite_connections"
        dendrite_connections = cortex[i].Dendrites_and_Synapses[0,:].astype(int)
        #print sample
        #print dendrite_connections
        #print sample[dendrite_connections]
        connected  = np.flatnonzero(sample[dendrite_connections])
        #print connected
        strength_of_synapse = cortex[i].Dendrites_and_Synapses[1,connected]
        #print strength_of_synapse
        connected_synapses = np.flatnonzero(strength_of_synapse > synaptic_perm_threshold)
        
        print connected_synapses
        cell_dendrite_num[i] = len(connected_synapses)
        #print cell_dendrite_num[i].astype(int)
    #print cell_dendrite_num.astype(int)
    cut_off = winner_cell_cutoff(cell_dendrite_num,top_conncetion_percent)
    top_cell_indices = np.flatnonzero(cell_dendrite_num > cut_off)
    synapse_inc=.05
    synapse_dec=.001
    #print cortex[top_cell_indices[0]].Dendrites_and_Synapses[1,0]
    adjusted_cells = adjust_synapse_strengths(cortex[top_cell_indices], synapse_inc, synapse_dec, synaptic_perm_threshold)
    cortex[top_cell_indices] = adjusted_cells
    #print cortex[top_cell_indices[0]].Dendrites_and_Synapses[1,0]

        
        
        
        #input_indices = np.where(sample[cortex[i].Dendrites_and_Synapses[0,:].astype(int)] > 1) 
        #print input_indices
        
        
        
        
        
        


[0 4 5]
[1 3 4 5 6]
[1 2 4 5 6 8]
[ 1  2  4  5  9 10]
[ 3  4  6  8  9 10]
[1 8]
[0 1 4 5 6 7 8 9]
[0 1 2 3 6 7 8]
[2 5 7 8 9]
[0 5 7]
[5 6 7 8]
[2]
[0 1 5 6 7]
[0 1 2 4 6 7]
[0 2]
[ 1  6  9 12]
[1 2]
[0 1 3 6]
[0 1 3 6]
[3 8]
[2 3 4 5 6 7]
[0 5]
[0 1 3 6 7 8 9]
[3 5]
[0 3 4]
[2 4 5]
[0 2 5 6]
[0 2 3 6]
[0 2 4]
[0 1 7]
[0 2 3 4 6 8]
[ 0  3  5  6  7  9 12 13]
[0 1 3 4 5 6]
[0 1 5]
[2]
[0 1]
[0 5 8]
[0 1 2 8]
[1 4 5]
[2 3 4 6 8 9]
[3 5]
[0 1 2 3]
[2 5 6]
[0 1 2 3 4 6 8]
[0 4 5 6 9]
[ 0  1  2  3  6  7  9 11]
[0 1 2 5 7 8 9]
[0 1 2 3]
[1 4 5 7 8 9]
[2 3 4 5 6 7]
[1 4 6 7]
[0 1 2]
[4 5]
[0 2 4 5 6]
[0 1 2 9]
[ 0  1  3  4  5  6  9 10]
[3 4 5 6]
[ 0  2  4  8  9 10 11 14]
[0 3]
[1 3 4]
[0 2 3 4 7 9]
[1 2 4 5 6 7 8]
[0 2 3 4 5 7]
[0 2 5]
[0 5 6]
[0 3]
[2 6 8]
[1 5 6 7 8]
[ 0  1  2  6  7  9 10]
[0 1 3 6]
[1 4]
[2 4 8]
[0 1 2 4 6]
[1 3 4 6 7]
[0 1 2 3 5 6]
[0 2 3 5 6 8]
[0 1 2 3 4 5 7 8 9]
[ 2  3  4  5  7  8  9 11]
[1 5]
[1 2 4 5 6 7]
[0 2 4]
[1 2 5 6 7 8 9]
[0 2 4 5]
[ 1  3  4  9 11]
[1 4 5 6 7]


[ 0  3  6  7  8 12]
[0 1 5]
[ 0  1  3  4  7  8 10]
[ 0  1  2  4  5  7 11]
[0 1 3 4 6 8]
[0 2 3 5 6 8]
[ 0  1  2  3  4  5  6  7 11]
[ 0  2  5  6  8 10 11 12]
[ 4  5  6  8  9 11 13]
[2 3 4]
[1 3]
[ 0  1  3  4  9 10]
[ 0  2  4  5  7  9 10 11]
[1 2 3 4]
[0 1 3 4 5 6 7]
[ 0  1  4  7  9 10]
[0 2 4 5 6 7]
[ 1  4  6 11]
[ 0  1  8 10 11 13]
[0 2 4 5 6 7]
[1 3 4 5 6 7 9]
[3 5 7 8]
[0 1]
[1 2 3 8]
[ 0  2  3  4  5  6  7 11]
[0 1 2 4 5 6 7]
[0 1 2 4 8]
[0 3 4 7]
[0 1 4 6]
[1 4 6 8]
[ 1  2  4  5 10 11]
[0 1 2]
[ 1  3  4  5  6  9 10 12]
[0 2 3 4]
[0 1 4]
[ 0  3  4  7  8  9 11]
[0 2 5 7 8]
[ 5  8 10]
[ 0  5  6 10 11]
[ 1  2  3  5  6  9 10 12]
[ 0  1  2  5  7  8  9 11]
[ 1  2  3  5  7 10 12 13 14]
[ 0  3  4  5  6  8 10 11 14 15]
[ 0  5  8  9 11 12 13]
[0 2 5 6 8]
[ 1  2  3  4 11 12]
[ 5  7  8 10 11 12]
[0 1 2 3 4 5 6 7 9]
[4 6]
[1 3 4 6 7]
[2 3 6 7 8]
[0 2 3 5 7 9]
[ 0  1  4  6  8 10 11 12 13 14]
[0 3 4 5 6 7 9]
[0 3 4]
[0 1]
[ 0  1  3  5  6  8 10 11]
[ 1  3  5  7  8  9 10 13]
[1 2 3 4]
[1 3 5 6]
[0 4 

[0]
[1 4 6 7]
[0 1 5 6]
[2 3 5 6 7]
[1 3 6 8]
[ 2  5  6  8  9 11 12 13]
[1 3 4 5 7]
[ 0  1  2  4  6  7 12]
[0 1 2 3 4 5 6]
[2 5 6 8]
[ 1  2  4  8  9 10]
[0 2 5 7]
[ 3  5  6  7 11]
[0 3 4]
[0 2 4 5]
[8]
[ 1  3  8  9 10 11]
[0 1 6]
[5 6]
[0 1 6 7 9]
[0 1 2 3 4 5 9]
[0 2 3 4 5 6]
[0 1 3 5 6 8 9]
[ 0  1  2  3  4  6 10]
[ 0  2  5  6  7  9 11 12]
[1 2 3 6]
[ 2  4  5  7  8 10]
[1 2 4 7]
[0 1 2 3 7 8 9]
[0 1 3 4 5 7 9]
[1 2 3 4 8]
[0 1 2 4 5 6 7]
[0 1 4 6 8]
[1 3 4 5]
[ 0  2  3  5 11]
[ 1  8  9 10]
[1 4 5 6 8 9]
[ 2  3  5  6  7 10]
[2 3 6 7]
[1 3 7 9]
[0 5]
[0 1 3 4 7]
[0 1 2 3 4 5 6 7 8]
[0 1 3 4 6 9]
[2 3 6]
[0 1 6]
[1 2 8 9]
[0 1 2 3 7 8 9]
[3 4 7]
[1 3 6 7]
[1 3 4 7 9]
[0 4 6 7]
[0 1 2 5 6 7]
[2 3 4 6 7 8]
[4 5 6 8]
[ 0  4  5  6  9 10]
[0 1 3 4 5]
[0 1 2 3 6 7]
[1 2 3 4 7 9]
[1 2 3 5 6 8 9]
[0 3 4 6 7]
[0 2 4 6 7]
[ 1  2  3  4  5  9 10 12 14 15 16]
[4 5 7 8 9]
[ 0  2  3  5  6  7 10 11]
[0 2 3 4 6]
[0 1 2 3 4 5 7]
[1 3]
[0 1 2 3 5 6 7 8]
[1 2 4 5 6 7]
[0 1 2 3 4 6]
[0 2]
[0 2]
[ 1  4  6  9 

[0 1 2 3 5 6]
[1 3 4 6 8]
[ 1  2  3  4  7  8 10 11]
[1 3 4 7]
[0 2 5]
[1 3 4 5]
[0 1 2 3 6]
[2 5 6 7]
[2 4 5]
[4 5]
[ 0  1  2  4  8 11 12]
[0 2 3 6]
[ 5  7  8 11 14]
[1 3 4 5 6 8]
[2 5 6]
[2 4 5]
[0 3 6]
[1 5 6 8]
[0 2 4 5 8]
[ 1  5  6  8  9 10]
[ 0  1  2  3  4  5  6  7  8 10]
[2 3 4 6]
[1 2 3 4 8]
[ 0  1  4  6  7  9 10 12]
[ 1  4  5  8  9 11]
[ 1  2  4  5  6  7  8 10 11]
[0 2 5]
[ 0  4  5  8  9 10 11]
[0 4]
[1 2 5 6 7 8]
[0 1 2 4 6]
[ 0  1  4  5  7  9 10]
[0 1 4]
[0 2 3 4]
[ 2  4  5  6  7  8  9 10]
[ 0  1  4  6  7  8 11 12 13]
[1 2 3 4 5 9]
[0 5 7]
[0 3]
[0 1 3 5 7 8]
[ 0  2  5  8  9 10]
[0 3 9]
[0 1 2 4]
[ 2  7  8 11]
[0 1 2 6 8]
[0 1 3 5]
[0 1]
[0 5 7 8]
[0 1 2 4 5 8]
[0 2 3 5 7]
[0 1 3 4 5 8 9]
[0 1 2 3 5]
[ 0  2  5  6  8  9 10]
[1 2 4 7 8]
[2 5 6]
[]
[ 0  1  2  3  5  7  9 10]
[1 2 3 6]
[1 3 4 8]
[0 2 4 6 7 8]
[1 2 3 4 5 7 8 9]
[1 2 5 7 8]
[1 4 6]
[0 6 7 8]
[0 2 4]
[2 6 8]
[0 2 3 6]
[0 1 4 5]
[ 3  5  6 11 12]
[0 1 2 3 4 6 7]
[0 1 2 3 4 5 6 7]
[ 1  2  3  4  6  7 10]
[2 4]
[0 1]
[ 1 

[]
[0 1 4 5]
[0 2 4 6 8]
[0 2 3 7]
[0 3 4 6]
[0 2 4 5]
[ 0  2  8  9 10]
[0 2 3 5 7 8 9]
[ 0  1  2  3  4  5  8  9 10]
[3 8]
[1 5]
[4]
[0 2 5 6 7 9]
[4 7]
[0 1 2 4]
[1 3 5 7]
[ 2  4  5  6  8 10 13]
[ 0  4  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  2  4  6  7 10 12 13]
[0 3 5]
[2 5]
[0 4 5 6]
[3 5 6]
[0 1 6]
[1 3 4]
[ 0  1  2  4  5  6  7 11]
[2 7]
[0 1 2 3 5 6 8]
[0 1 2 5]
[0 1 2 5]
[0 1 4 5 8]
[0 1 2 3 4 6 8 9]
[3 5]
[0 1 2 5 9]
[1 3]
[0 1 2 3]
[0 1 2 5 7]
[0 2 5 6]
[0 1 3 4 6 8 9]
[1 2 4 6 7 9]
[ 0  2  3  7 10 11]
[0 1 2]
[1 2 4 7 8]
[ 0  2  7  9 10]
[2 3 4 5 6 7 8 9]
[ 1  3  4  7 10 12]
[5 6 7]
[ 2  4  6  7  8  9 12]
[0 1 3 5 7 8 9]
[0 1 5 6 7 8]
[ 0  1  2  3  4  5  7  9 11]
[0 2 3 4 8]
[1 2 3 4 6 7 9]
[1 2 3 7]
[0 2 3 6 7 8]
[ 1  3  4  6 10 11 13]
[0 1 3 6]
[ 4  5  7  9 10]
[3 5 6 7]
[ 3  4  5  7  9 10 11]
[0 1 2]
[0 1 2 4]
[2 3 4]
[4 5 6]
[2 6 7 8]
[0 3]
[0 3 4 7 9]
[0 2 4 7 9]
[2 3 5 6 8]
[2 3 4 5 7 8]
[0 1 3 4 5 6]
[ 1  2  7  9 10]
[0 1 2 3 4 5 6]
[2]
[ 2  3  6  7  9 10]


...

[0 1 2 3]
[0 1 2 3 5 8 9]
[0 1 4]
[1 3 5 7]
[ 1  4  9 10]
[1]
[ 1  2  3  4  8  9 10]
[1 3 4]
[ 1  2  3  8  9 10 11 12]
[2 3 6]
[0 1 3 5 6 7 8 9]
[2 4 5]
[1 2 3 4 5 7 8]
[0 1 2 3 6 7]
[2 3 4 7]
[6]
[1 3 4 5 6 7]
[0 3 6 9]
[0 2 5 7 8]
[3 4 8]
[ 1  2  4  8 10]
[7 8 9]
[0 2]
[0 3 4 5]
[ 1  3  6  7  8  9 10]
[1 3 4 5 7 8 9]
[1 2 3]
[0 1 3 4 7 8]
[ 0  1  2  3  4  8 10 11]
[0 1 3 4 7 8]
[ 1  2  4  8  9 10]
[3 5 6 7 8]
[1 4 6 7 8 9]
[1 4 5 7]
[0 7]
[0 2]
[0 1 2 4 5 6 7 8]
[ 0  4  9 10 11]
[2 3 4]
[0 2 4 5 7 8]
[ 0  3  4  5  7  9 10]
[ 1  2  3  4  5  6  9 10]
[0 2 5 6]
[1 2 3 5 6]
[ 1  2  3  4  5  6 10]
[3 5 6]
[2 4 5]
[0 1 2 3 4]
[ 1  2  6  7  8  9 10]
[0 3 5 7 8]
[1]
[ 0  1  2  6  8  9 10]
[1 4 5]
[0 1 3]
[2 3 4 6 7]
[0 2 7 8]
[0 1 2 3 4 5]
[3 6]
[0 2 3 4]
[0 2 4 5 6 7 8]
[0 1 2 3 4]
[2 3 4 5 6 7]
[1 3 4 5 6]
[0 1 4 7 9]
[1 2 4 6]
[2 3 6 7 8 9]
[ 0  3  6 10 11 12]
[1 3 5]
[1 2 8]
[ 1  4 10 11 12 13]
[ 2  5  7  8  9 10]
[1 2 5 6]
[0 1 6 7 8]
[0 3 5 6 7 8]
[0 1 3 6 7 8]
[1 2 4]
[ 4  7  8  9 10 

[0 1 6 7]
[0 3 4 9]
[ 0  4  5 10 11]
[2 5 6]
[0 1 2 3 4 7 8]
[2 3 4 5 6]
[0 7 8]
[1 3 6 8]
[1 2 3 5 7]
[1 3 5 6 7]
[1 4 6 7]
[0 1 4]
[0 3 5 6 7 8 9]
[4 5 6]
[0 1 4 7 8 9]
[ 0 10 11]
[0 1 2]
[ 3  4  5 10]
[ 1  2  4  6  7 10]
[0 1 2 7 8]
[0 1 2]
[ 1  2  3  4  5  6  7  9 10]
[ 0  3  7  8  9 11 12]
[2 6 7 8 9]
[1 2 3 4 5 6]
[0 1 2 3]
[ 1  2  3  4  5  6  9 10 11 12]
[0 2 4 7 8 9]
[2 3 5 7 9]
[ 6  8  9 10]
[ 1  3  4  9 10 11 12]
[2 6 8]
[2 6 7]
[0 4 8 9]
[ 0  3  5  6  7  9 10 11]
[2 4 7 8]
[0 2 3 5 6]
[0 2 3 6]
[0 2 4]
[0 2 4 5]
[ 2  3  6  8 11 12]
[ 1  2  3  4  8  9 10]
[2 3 4 5 6 8]
[0 3 6]
[ 2  4  6  7  8  9 10 12]
[0 1 3 4 7 8 9]
[0 3 6 7 8]
[1 4 6 8 9]
[0 1 2]
[ 2  3  6  9 10]
[ 3  6  9 10 11 12 13]
[1]
[2 3 6]
[ 0  2  4  7  9 10 11 12]
[0 1 2 4 6 7 8 9]
[ 4  6  8 10]
[0 2 4 5 6 7 8]
[2 3 4 6]
[0 1 2 4 6]
[1 2 4 8 9]
[ 1  5  6  7  8 10]
[ 0  1  2  3  4  7  9 11 12]
[ 1  3  6  8 10]
[ 2  3 10 11]
[4 5 6 7 8]
[ 0  2  5  6  8 11]
[1 2 7 8]
[ 1  8  9 10]
[0 2 3 4 5]
[2 3 4 6 7 8]
[0 1 4]
[ 

[0 3 4 5 7 8]
[2 4 5]
[0 1 2 3 6 8 9]
[2 3 6 7 9]
[1 6]
[3 5 7]
[0 1 3]
[ 0  2  9 10]
[0 2 3 4 5 6 7 9]
[0 3 4 5]
[2 4 5 6]
[2 6]
[0 2 4 5 6 9]
[1 5 7 8]
[0 2 6 8]
[0 2 6 7]
[1]
[0 1 5 7 8]
[0 1 2 6]
[0 1 4 5 7]
[0 4 5 6 8 9]
[1 2 3 5 6]
[1 2 6 9]
[0 1 2 3 4 6 7 8 9]
[0 1 3 5 6]
[ 0  1  3  4  6  7 11 12 13]
[3 4 6]
[ 0  8  9 10 11]
[ 0  3  6 10 11 12]
[1 4 5 6]
[0 1 2 9]
[ 1  2  3  6  7 10 11]
[ 0  1  2  5  7  8 11 12 13]
[ 0  2  4  8  9 10]
[ 4  7  8 10 11 12]
[ 0  2  5  6  7 10 13 14]
[0 2 4 5 7 8]
[1 2 3 6]
[ 5  8 10 11]
[0 1 2 3]
[1 3 5 6]
[5]
[2 3]
[2 3 4 6 7 8]
[0 3 4 5 6]
[0 1 3 5 6 7 8 9]
[ 3  4 12 13]
[1 2 6 7 9]
[0 5 7]
[0 1 3 4 7]
[2 7]
[0 1 3 4 9]
[1 2 3 5]
[3 6]
[0 1 2 3 4 6]
[0 1 4 5 7 8 9]
[1 2 3 7]
[ 2  3  4  6  7 11]
[0 2 5 7 9]
[0 1 2 4 6 7 8 9]
[ 0  4  7  8 10]
[0 2 4 5]
[0 1 2]
[0 2 3 4 5]
[ 1  2  4  8 11]
[ 1  3  4 10 11 12]
[0 3 5 6 7 8 9]
[1 4 5 7 8 9]
[1 3 9]
[0 1 3 4 6 8 9]
[0 1 2 4 5 6 7 9]
[ 0  2  3  5  6  8 11]
[ 2  5  8  9 11 12 13]
[0 1 2 5 6 7]
[0 3 4 6]


[0 3 4 7 8]
[2 4 6]
[ 1  2  4  5 11]
[6 7]
[3 4 5 7]
[5 6]
[ 1  3  4  5  6  7  9 11 12 13]
[ 2  8 12 13]
[1 2 3]
[ 0  1  2  3  6  7 11 13]
[1 3 6 8]
[0 3 4 5 6 9]
[1 2 5 7]
[0 1 3 6 7]
[ 0  2  3  4  5  8  9 10 13]
[0 1 3 4 5 6]
[ 2  4  7  8  9 10 11]
[ 0  2  3  7  8 10 11]
[1 2 6 7]
[2 4 8]
[3 5 6 7]
[1 2 3 5 7]
[0 1 2 6]
[ 2  3  5  8  9 10]
[ 2  5  7 10 11 12]
[ 0  1  2  3  4  5 12 13]
[ 0  4  8 10]
[ 2  3  4  6  9 10 11 13 14]
[ 0  1  2  5  6  7 10 11]
[1 3 5]
[0 1 2 4]
[4 5 6 8 9]
[ 0  7  8  9 12 13]
[0 2 3 4 5 7]
[ 1  3  6  9 10]
[ 1  4  5 10 11]
[0 1 2 6]
[ 0  3 10 13]
[ 0  3  4  5  6  7  8  9 10 11]
[2 3 4 5 6 7 8]
[0 1 2 3 4 7]
[0 4 5 7 8 9]
[ 1  4  6  7 10 12]
[ 0  3  6  7 10 11 12]
[ 2  3  4  5  8 10 11 13]
[ 2  3  4  5  7 10]
[ 1  2  5  6  7  8 11]
[ 1  3  6  7  8 12]
[ 2  4  5  8 10]
[1 3 4 5 8 9]
[1 2 4 6 7 9]
[4 6]
[1 2 4 5 7]
[1 4 5 6]
[1 3 5 7]
[0 2 3]
[1 4 5 6 8 9]
[ 0  2  5  8  9 10 11]
[4 5 6 7]
[0 2 3 4 6]
[ 0  1  2  5  6  8 10 11]
[ 0  2  5  9 12]
[1 2 3 4 6 8]
[ 0 

KeyboardInterrupt: 

In [24]:
# def Classification(input_sample,cortex,synaptic_perm_threshold):
#predition = np.zeros(len(input_sample))
synaptic_perm_threshold = 0.5
top_percent = .50
actual = sample_input[48]
on_bits = np.flatnonzero(actual > 0)

cummulative_array = []
for i in range(len(cortex)):
    #find dendrite connections to input
    dendrite_connections = cortex[i].Dendrites_and_Synapses[0,:].astype(int)
    #find dendrites connected to on bits in input
    connected  = np.flatnonzero(sample[dendrite_connections])
    #find number of dendrites for cell that are over threshold
    strength_of_synapse = cortex[i].Dendrites_and_Synapses[1,connected]
    connected_synapses = np.flatnonzero(strength_of_synapse > synaptic_perm_threshold)
    input_connected_addresses = cortex[i].Dendrites_and_Synapses[0,connected_synapses].astype(int)
    cummulative_array = np.concatenate((cummulative_array,input_connected_addresses)).astype(int)
sum_array = np.bincount(cummulative_array)

prediction_indices = (-sum_array).argsort()[:10]

num_overlap = len(np.intersect1d(on_bits, prediction_indices))

print num_overlap

probability = float(num_overlap)/len(on_bits)

print probability






    
    







7
0.0376344086022
